### Reinforcement Learning - Resource Manager

Kudos to:

https://www.gymlibrary.dev/content/environment_creation/

https://www.youtube.com/watch?v=bD6V3rcr_54&ab_channel=NicholasRenotte 

Version 1.0:

- Export the Environment to python file
    - Register it
- Add Deep Learning with DQN



### Imports

In [1]:
import gym
from gym import spaces
import numpy as np
import pygame
from gym.envs.registration import register
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Environment

In [2]:
from ResourceManagerEnvironment import ResourceManagerEnv

In [3]:
#register the environment
register(
    id='Resource-Manager-v1-1',
    entry_point='ResourceManagerEnvironment:ResourceManagerEnv',
    max_episode_steps=300,
)


In [4]:
env = gym.make('Resource-Manager-v1-1')

c:\Users\denni\.conda\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (10, 10)
  logger.warn(


## Deep Learning

In [8]:
#define grid size before training
new_grid_size = 15

In [9]:
class DQN(tf.Module):
    def __init__(self, num_actions):
        self.model = keras.Sequential([
            keras.layers.Input(shape=(new_grid_size, new_grid_size)),
            keras.layers.Flatten(),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(num_actions)  # Output layer with num_actions units
        ])

    def call(self, state):
        return self.model(state)



In [10]:
class DQNAgent:
    def __init__(self, num_actions, epsilon=1.0, epsilon_decay=0.995, min_epsilon=0.01, learning_rate=0.001, gamma=0.99):
        self.q_network = DQN(num_actions)
        self.target_network = DQN(num_actions)
        self.optimizer = keras.optimizers.Adam(learning_rate)
        self.num_actions = num_actions
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.gamma = gamma

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.num_actions)
        q_values = self.q_network(state[None, :])
        return np.argmax(q_values[0])

    def train(self, state, action, reward, next_state, done):
        with tf.GradientTape() as tape:
            q_values = self.q_network(state)
            target_q_values = self.target_network(next_state)
            target = q_values.numpy()
            if done:
                target[0][action] = reward
            else:
                target[0][action] = reward + self.gamma * np.max(target_q_values)

            loss = tf.reduce_mean(tf.square(target - q_values))
        gradients = tape.gradient(loss, self.q_network.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.q_network.trainable_variables))

    def update_target_network(self):
        self.target_network.set_weights(self.q_network.get_weights())

    def decay_epsilon(self):
        if self.epsilon > self.min_epsilon:
            self.epsilon *= self.epsilon_decay


In [13]:

env = gym.make('Resource-Manager-v1-1', grid_size=new_grid_size, render_mode='human')
episodes = 100  # Number of episodes
max_steps = 300  # Maximum steps per episode

# Initialize the agent
num_actions = env.action_space.n
dqn_agent = DQNAgent(num_actions)

for episode in range(episodes):
    state = env.reset()
    done = False

    for step in range(max_steps):
        action = dqn_agent.choose_action(state)
        step_result = env.step(action)

        next_state = step_result[0]
        reward = step_result[1]
        done = step_result[2]
        info = step_result[3]

        #next_state, reward, done, info = step_result
        dqn_agent.train(state, action, reward, next_state, done)
        state = next_state

        if done:
            break

    dqn_agent.decay_epsilon()
    dqn_agent.update_target_network()


TypeError: 'DQN' object is not callable